In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, LayerNormalization, Add, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam, AdamW
from keras.regularizers import l2
from tensorflow.keras import backend as K
from random import choice
from xgboost import XGBRegressor

# --- データ準備 ---
df = pd.read_csv("../datasets/ring_data.csv")

# 特徴量作成（ラグ・移動統計など）
df['体温_lag1'] = df['体温'].shift(1)
df['室温_lag1'] = df['室温'].shift(1)
df['delta_temp'] = df['室温'] - df['体温']
df['delta_temp_lag1'] = df['室温_lag1'] - df['体温_lag1']
df['体温_ma3'] = df['体温'].rolling(window=3).mean()
df['体温_var3'] = df['体温'].rolling(window=3).var()
df['室温_ma3'] = df['室温'].rolling(window=3).mean()
df['室温_var3'] = df['室温'].rolling(window=3).var()
df.dropna(inplace=True)

# 特徴量定義
features = [
    '体温', '体温_lag1',  '体温_ma3', '体温_var3', 
    '室温', '室温_lag1',  '室温_ma3', '室温_var3', 
    'delta_temp', 'delta_temp_lag1',  
    'アンケート'
]
target = '適温'

# データ抽出
X_raw = df[features].values
y_raw = df[[target]].values

# ノイズによるデータ拡張
noise_std = 0.01 * np.std(X_raw, axis=0)
noise = np.random.normal(loc=0.0, scale=noise_std, size=X_raw.shape)
X_aug = X_raw + noise
y_aug = y_raw.copy()
X_combined = np.vstack([X_raw, X_aug])
y_combined = np.vstack([y_raw, y_aug])

# スケーリング
feature_scaler = MinMaxScaler()
X_scaled = feature_scaler.fit_transform(X_raw)

target_scaler = MinMaxScaler()
y_scaled = target_scaler.fit_transform(y_raw)

# シーケンス生成関数
def create_sequences(X, y, time_steps=3):
    n_samples = len(X) - time_steps
    X_seq = np.array([X[i:i+time_steps] for i in range(n_samples)])
    y_seq = y[time_steps:]
    return X_seq, y_seq

# シーケンス生成
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps = 3)

# データ分割（shuffle=False）
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False, random_state=42)

# --- LSTM モデル学習 ---
epochs_list = [60, 90, 120]
batch_sizes = [4, 8, 16]
lstm_units = [60, 90, 120]
dropout_rates = [0.001, 0.01, 0.05]
l2_reg_strengths = [0.01, 0.1, 0.2]
best_loss = float('inf')
best_model = None

for _ in range(15):
    epochs = choice(epochs_list)
    batch_size = choice(batch_sizes)
    units = choice(lstm_units)
    dropout = choice(dropout_rates)
    l2_reg_strength = choice(l2_reg_strengths)

    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = Bidirectional(LSTM(units, return_sequences=True, kernel_regularizer=l2(l2_reg_strength)))(input_layer)
    x = LayerNormalization()(x)
    x = Bidirectional(LSTM(units // 2, return_sequences=False, kernel_regularizer=l2(l2_reg_strength)))(x)
    x = LayerNormalization()(x)

    residual = x
    residual_units = K.int_shape(residual)[-1]
    x = Dense(residual_units, activation='gelu', kernel_regularizer=l2(l2_reg_strength))(x)
    x = Add()([x, residual])
    x = LayerNormalization()(x)
    x = Dropout(dropout)(x)

    output = Dense(1)(x)
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), loss='mse')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=0)

    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )

    val_loss = history.history['val_loss'][-1]
    if val_loss < best_loss:
        best_loss = val_loss
        best_model = model
        best_history = history  # ← ここで保存

# --- LSTM特徴抽出 ---
feature_extractor = Model(inputs=best_model.input, outputs=best_model.layers[-3].output)
X_train_lstm_feat = feature_extractor.predict(X_train)
X_test_lstm_feat = feature_extractor.predict(X_test)

# --- XGBoost用に元データと同期 ---
time_steps = 3
X_raw_cut = X_raw[time_steps:]
y_raw_cut = y_raw[time_steps:]
X_train_static = X_raw_cut[:len(X_train)]
X_test_static = X_raw_cut[-len(X_test):]
X_train_combined = np.hstack([X_train_static, X_train_lstm_feat])
X_test_combined = np.hstack([X_test_static, X_test_lstm_feat])

# --- XGBoost 学習・予測 ---
xgb_model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train_combined, y_train)

xgb_preds = xgb_model.predict(X_test_combined)
xgb_preds_inv = target_scaler.inverse_transform(xgb_preds.reshape(-1, 1)).flatten()
actual_temps_inv = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# 精度指標計算
mse = mean_squared_error(actual_temps_inv, xgb_preds_inv)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actual_temps_inv, xgb_preds_inv)
r2 = r2_score(actual_temps_inv, xgb_preds_inv)

print(f"XGBoost Mean Squared Error (MSE): {mse:.4f}")
print(f"XGBoost Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"XGBoost Mean Absolute Error (MAE): {mae:.4f}")
print(f"XGBoost R^2 Score: {r2:.4f}")


sns.set(style="whitegrid", palette="muted", font_scale=1.2)

# --- LSTMのトレーニングと検証損失推移 ---
plt.figure(figsize=(12, 7))
sns.lineplot(data=best_history.history['loss'], label='トレーニング損失', color='royalblue', linewidth=2)
sns.lineplot(data=best_history.history['val_loss'], label='検証損失', color='tomato', linewidth=2)
plt.title('トレーニングと検証の損失推移')
plt.xlabel('エポック')
plt.ylabel('損失 (MSE)')
plt.legend()
plt.tight_layout()
plt.show()

# --- ハイブリッドモデル予測と実測の適温推移グラフ ---
plt.figure(figsize=(12, 7))
sns.lineplot(x=range(len(xgb_preds_inv)), y=xgb_preds_inv, label='ハイブリッドモデル予測適温', color='royalblue', linewidth=2)
sns.lineplot(x=range(len(actual_temps_inv)), y=actual_temps_inv, label='実際の適温', color='tomato', linestyle='--', linewidth=2)
plt.title('ハイブリッドモデル予測適温と実際の適温の比較')
plt.xlabel('時間ステップ')
plt.ylabel('温度 (℃)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
import os


# 評価指標を辞書でまとめる
results_dict = {
    "MSE": [mse],
    "RMSE": [rmse],
    "MAE": [mae],
    "R2": [r2]
}

# DataFrameに変換
results_df = pd.DataFrame(results_dict)

# 保存先ディレクトリ
results_dir = "../results"
os.makedirs(results_dir, exist_ok=True)  # フォルダがなければ作成

# 保存ファイル名
filename = "ring_lstm_xgb.csv"  

# フルパスを作成
filepath = os.path.join(results_dir, filename)

# CSV保存（indexは不要）
results_df.to_csv(filepath, index=False)

print(f"評価結果を保存しました: {filepath}")
